In [1]:
# импортируем необходимые библиотеки
import pandas as pd
from scipy import stats

In [2]:
# читаем файл, оставляем только нужные столбцы
df = pd.read_csv('case_ctr.csv', sep=';')
df = df[['user_id', 'session_id', 'click_id', 'experimentVariant']]
df.head()

,user_id,session_id,click_id,experimentVariant
0,4299b8c9c50c3e4e8a067f448426463e,08f9c3b0e2ff8da7bad6cc200fa735a2,56bcee80ce68ddc58c178a63288965b2,1
1,863829e0fc8e0dcd88f44c6a63db836a,d0ebf6ddaa8702e6de4e46854a1136ff,b209c6b14e91606a99d637ff6eb74cc6,1
2,640af3312483bea44b8159a8caaf0b3d,838c4a718830b5f1ac7be5e70745ff47,3f3d510c709ae5d9bae0331fcf776f3a,1
3,43de481cae63944b81db6fd5567a12bb,8f36aead1ceafb3e67f9957a7765549e,da73306047217a73a6ee0ea7477f2ad7,0
4,b84dd1a6c3df2e08eb7bef496cdf6d39,09799cdf1394b0e4b86ace899b7f218e,6e4b93932be600a5536c291888996037,1


In [3]:
# информация о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            10000 non-null  object
 1   session_id         10000 non-null  object
 2   click_id           7655 non-null   object
 3   experimentVariant  10000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [4]:
# смотрим распредение по группам в эксперименте
df.experimentVariant.value_counts()

0    5112
1    4888
Name: experimentVariant, dtype: int64

In [5]:
# создаём датафрейм с контрольной и тестовой группой
df_control = df.query('experimentVariant == 0')
df_test = df.query('experimentVariant == 1')

In [6]:
# проверка на то, что один и тот же пользователь существует в двух группах эксперимента. 
# если True, значит пользователи не пересекаются

print(df.user_id.nunique() == df_test.user_id.nunique() + \
                        df_control.user_id.nunique())

# проверка на то, что одна и та же сессия существует в двух группах эксперимента
# если True, значит сессии не пересекаются
print(df.session_id.nunique() == df_test.session_id.nunique() + \
                            df_control.session_id.nunique())

True
True


In [7]:
# группируем данные в контрольной группе по user_id с подсчетом кол-ва сессий и кликов
df_control_us_uniq = df_control.groupby('user_id', as_index = False).agg({'session_id' : 'count', 'click_id' : 'count'})
df_control_us_uniq.columns = ['user_id', 'session_count', 'click_count']
df_control_us_uniq['ctr'] = round(df_control_us_uniq.click_count / df_control_us_uniq.session_count * 100,2)

# проверяем может ли показатель CTR в контрольной группе быть больше чем 100%
display(df_control_us_uniq.sort_values('ctr', ascending = False).head(1))

avg_ctr_control = round(df_control_us_uniq.ctr.mean(),2)
print('Средний показатель CTR в тестовой группе = {}%'.format(avg_ctr_control))

,user_id,session_count,click_count,ctr
564,80526421c5acf148d0fd2e788eb0d819,2,2,100.0


Средний показатель CTR в тестовой группе = 77.23%


In [8]:
# группируем данные в тестовой группе по user_id с подсчетом кол-ва сессий и кликов
df_test_us_uniq = df_test.groupby('user_id', as_index = False).agg({'session_id' : 'count', 'click_id' : 'count'})
df_test_us_uniq.columns = ['user_id', 'session_count', 'click_count']
df_test_us_uniq['ctr'] = round(df_test_us_uniq.click_count / df_test_us_uniq.session_count * 100,2)

# проверяем может ли показатель CTR в тестовой группе быть больше чем 100%
display(df_test_us_uniq.sort_values('ctr', ascending = False).head(1))

avg_ctr_test = round(df_test_us_uniq.ctr.mean(),2)
print('Средний показатель CTR в тестовой группе = {}%'.format(avg_ctr_test))

,user_id,session_count,click_count,ctr
1116,ffc1dd3e8f3c8594ef3f7b4423cb3b6b,2,2,100.0


Средний показатель CTR в тестовой группе = 75.83%


In [9]:
print('CTR в тестовой группе отличается на {}% от CTR в контрольной группе'.format(round(avg_ctr_test - avg_ctr_control, 2)))

CTR в тестовой группе отличается на -1.4% от CTR в контрольной группе


In [10]:
print(stats.mannwhitneyu(df_control_us_uniq.ctr, df_test_us_uniq.ctr))
print(stats.ttest_ind(df_control_us_uniq.ctr, df_test_us_uniq.ctr))

MannwhitneyuResult(statistic=609338.5, pvalue=0.07842795146979505)
Ttest_indResult(statistic=1.3866226998396367, pvalue=0.16569457697014517)


Из проведенного выше теста видим, что изменение в показателе CTR в контрольной и тестовой группах отличаются, но это отличие статистически не значимо (pvalue > 0.05)